# Predicting Pulsar Stars
#### Use the HTRU 2 dataset to predict pulsars.

In [ ]:
#installs
!pip install jovian --upgrade --quiet
#Imports
import jovian
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, TensorDataset
from torchvision.utils import make_grid
from torchvision.datasets.utils import download_url
import zipfile
%matplotlib inline

In [ ]:
project_name="Predicting-Pulsars" #Project name to commit to on Jovian.ml

## Download the dataset and initialize the dataset.
 - if you are on kaggle, add [this](https://www.kaggle.com/pavanraj159/predicting-a-pulsar-star) dataset- avoids redownloading the dataset each time.
 - Otherwise uncomment the next two lines of code.

In [ ]:
#data_url="https://archive.ics.uci.edu/ml/machine-learning-databases/00372/HTRU2.zip"
#download_url(data_url, ".")

#### Extract the csv file - if you downloaded the dataset.

In [ ]:
# Uncomment as needed.
#with zipfile.ZipFile("./HTRU2.zip", 'r') as zip_ref:
#    zip_ref.extractall(".")
#!rm -rf HTRU2.zip

### Load the data from the .csv file 
We just need to use the panda library's read_csv() function


In [ ]:
filename = "../input/predicting-a-pulsar-star/pulsar_stars.csv" #Change as needed.

df = pd.read_csv(filename)
df.head()

## Prepare Dataset for Training
We need to convert the dataframe to Pytorch Tensors using numpy arrays. 

In [ ]:
# This function converts into a numpy array
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

In [ ]:
inputs_df=df.drop("target_class",axis=1)#Easiest way to get inputs- we just need everything but the targets_class 
inputs_arr=inputs_df.to_numpy()
targets_df=df["target_class"]#Easiest way to get outputs-need just targets_class
targets_arr=targets_df.to_numpy()

In [ ]:
#output variables.
inputs=torch.from_numpy(inputs_arr).type(torch.float64)
targets=torch.from_numpy(targets_arr).type(torch.int16)

### Commit to Jovian(optional)

In [ ]:
#jovian.commit(project=project_name, enviroment=None)

### Create the dataset.

In [ ]:
dataset=TensorDataset(inputs, targets)

### Split the dataset into training and validation

In [ ]:
num_rows=df.shape[0]
val_percent = .1 # How much of the dataset 
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size

Use the random_split function to split dataset into 2 parts of the desired length

In [ ]:
train_ds, val_ds = random_split(dataset, (train_size, val_size))

### Set a batch size.  
I am going to pick 200, but adjust this to you needs.

In [ ]:
batch_size=200

## Load Data and transfer data to GPU, if available.

In [ ]:
# PyTorch data loaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=3, pin_memory=True)

### Transfer to GPU

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
# get device
device=get_default_device()
device

We can now wrap our training and validation data loaders using DeviceDataLoader for automatically transferring batches of data to the GPU (if available).


In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

## Create a Model
We are going to be creating a Model with Residual Blocks and Batch Normalization

### Residual Block

In [ ]:
class SimpleResidualBlockLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.linear()
        self.sigmoid1 = F.sigmoid()
        self.linear2 = nn.linear()
        self.sigmoid1 = F.sigmoid()
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.sigmoid1(out)
        out = self.conv2(out)
        return self.sigmoid1(out) + x # Non-linear can be applied before or after adding the input0

In [ ]:
class HTRU2Model(nn.Module):
    def __init__(self):
        super.__init__()
        self.res_block1=SimpleResidualBlockLogistic()
        self.res_block2=SimpleResidualBlockLogistic()




In [ ]:


@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history
